In [8]:
# Illegal Dumpings Test Script

import pandas as pd
import requests
from datetime import datetime
import json

# Step 1: Fetch current dataset from GitHub
url = "https://raw.githubusercontent.com/jeisey/phiti/main/graffiti.csv"
current_data = pd.read_csv(url, parse_dates=['requested_datetime', 'closed_datetime'], encoding='latin1')

# Step 2: Find the most recent requested_datetime
latest_date = current_data['requested_datetime'].max()
latest_date = latest_date.tz_convert('UTC')

# Step 3: Query the API for new or modified records
query = f"""
SELECT cartodb_id,objectid,service_request_id,subject,status,status_notes,requested_datetime,updated_datetime,expected_datetime,closed_datetime,address,zipcode,media_url,lat,lon 
FROM public_cases_fc 
WHERE 
      ( 
        (requested_datetime > '2025-07-01 00:00:00+00:00') OR 
        (status = 'Open' AND closed_datetime IS NOT NULL)
      ) 
      AND subject = 'Illegal Dumping'
      AND media_url IS NOT NULL
      AND media_url <> ''
"""
response = requests.get("https://phl.carto.com/api/v2/sql", params={'q': query})
new_data = pd.DataFrame(response.json()['rows'])
# Check if there are new records, if not then exit
if new_data.empty:
    print("No new records fetched from the API.")
    sys.exit(0)  # Exit the script gracefully with a status code of 0 (normal termination)

new_data['requested_datetime'] = pd.to_datetime(new_data['requested_datetime'])

# Step 4: Perform upsert operation
# Update modified records
mask = (current_data['status'] == 'Open') & (current_data['closed_datetime'].isna())

In [7]:
new_data

,cartodb_id,objectid,service_request_id,subject,status,status_notes,requested_datetime,updated_datetime,expected_datetime,closed_datetime,address,zipcode,media_url,lat,lon
0,2003826,5306844,17825185,Illegal Dumping,Open,None,2025-06-30 17:21:05+00:00,2025-07-02T17:59:39Z,2025-07-08T00:00:00Z,None,3401 JUDSON ST,19140,https://d17aqltn7cihbm.cloudfront.net/uploads/...,40.007241,-75.167417
1,2003912,5307009,17825697,Illegal Dumping,Open,None,2025-06-30 18:10:09+00:00,2025-07-02T18:21:44Z,2025-07-08T00:00:00Z,None,700 N 39th St,19104,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.966016,-75.200859
2,2003803,5306821,17825136,Illegal Dumping,Open,None,2025-06-30 17:13:36+00:00,2025-07-02T17:03:07Z,2025-07-08T00:00:00Z,None,712 RODMAN ST,19147,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.942626,-75.154335
3,2003810,5306828,17825148,Illegal Dumping,Open,None,2025-06-30 17:15:36+00:00,2025-07-02T19:03:15Z,2025-07-08T00:00:00Z,None,4601 N BROAD ST,19140,https://d17aqltn7cihbm.cloudfront.net/uploads/...,40.023078,-75.147935
4,2003923,5307020,17825717,Illegal Dumping,Open,None,2025-06-30 18:12:41+00:00,2025-07-02T18:21:44Z,2025-07-08T00:00:00Z,None,2850 JUDSON ST,19132,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.998772,-75.169663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023093,5328641,17869804,Illegal Dumping,Open,None,2025-07-02 19:47:45+00:00,2025-07-02T19:52:50Z,2025-07-10T00:00:00Z,None,1540 S CARLISLE ST,19146,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.930903,-75.169268
86,2023101,5328648,17869814,Illegal Dumping,Open,None,2025-07-02 19:51:47+00:00,2025-07-02T19:56:55Z,2025-07-10T00:00:00Z,None,2710 55th Dr,None,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.929941,-75.215075
87,2023361,5328944,17870190,Illegal Dumping,Open,None,2025-07-02 23:51:59+00:00,2025-07-02T23:57:05Z,2025-07-10T00:00:00Z,None,1240 S 11TH ST,19147,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.933612,-75.162436
88,2023364,5328947,17870193,Illegal Dumping,Open,None,2025-07-03 00:04:10+00:00,2025-07-03T00:09:17Z,2025-07-10T00:00:00Z,None,2220 CASTOR AVE,19134,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.997219,-75.097842


In [9]:
new_data

,cartodb_id,objectid,service_request_id,subject,status,status_notes,requested_datetime,updated_datetime,expected_datetime,closed_datetime,address,zipcode,media_url,lat,lon
0,2010489,5308430,17835592,Illegal Dumping,Open,None,2025-07-01 00:14:25+00:00,2025-07-02T20:43:45Z,2025-07-08T00:00:00Z,None,861 N 26TH ST,19130,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.972232,-75.179132
1,2010564,5308549,17836618,Illegal Dumping,Open,None,2025-07-01 00:33:47+00:00,2025-07-02T20:43:49Z,2025-07-08T00:00:00Z,None,912 LOCUST ST # A,19107,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.947164,-75.156778
2,2010792,5308962,17839658,Illegal Dumping,Open,None,2025-07-01 01:35:05+00:00,2025-07-02T20:43:49Z,2025-07-08T00:00:00Z,None,1113 FITZWATER ST # A,19147,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.941248,-75.161180
3,2014882,5312726,17852523,Illegal Dumping,Open,None,2025-07-01 11:42:23+00:00,2025-07-02T21:03:12Z,2025-07-08T00:00:00Z,None,4733 WALNUT ST,19139,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.956074,-75.216916
4,2012043,5309032,17840293,Illegal Dumping,Open,None,2025-07-01 01:47:34+00:00,2025-07-02T15:41:32Z,2025-07-08T00:00:00Z,None,323 S 13th St,19107,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.945909,-75.162712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2023396,5328980,17870234,Illegal Dumping,Open,None,2025-07-03 00:55:07+00:00,2025-07-03T01:00:14Z,2025-07-10T00:00:00Z,None,3900 PALMETTO ST,19124,https://d17aqltn7cihbm.cloudfront.net/uploads/...,40.006998,-75.108358
63,2023397,5328981,17870237,Illegal Dumping,Open,None,2025-07-03 01:31:22+00:00,2025-07-03T01:36:35Z,2025-07-10T00:00:00Z,None,1904–1998 S Eighth St,19148,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.924409,-75.159489
64,2023402,5328986,17870250,Illegal Dumping,Open,None,2025-07-03 02:34:35+00:00,2025-07-03T02:39:41Z,2025-07-10T00:00:00Z,None,3279 RED LION RD,19114,https://d17aqltn7cihbm.cloudfront.net/uploads/...,40.079835,-74.991729
65,2023410,5328994,17870263,Illegal Dumping,Open,None,2025-07-03 03:10:27+00:00,2025-07-03T03:15:33Z,2025-07-10T00:00:00Z,None,1410 S 6TH ST,19147,https://d17aqltn7cihbm.cloudfront.net/uploads/...,39.930913,-75.154897


In [10]:
new_data.to_csv('ill_dumpings.csv', index=False)